In [1]:
# default_exp datapipe
from nbdev import *

# Database - functions for data back-end / manipulations

This is using an alternate approach:
  - I have exported all of my Apple healthfit data from the Health app to export.zip 
  and then converted this to a SQLite database using `healthfit-to-sqlite`
  - I am then creating a "published" version of this SQLite database using
  Datasette.io
  - I have a local version of this database running at http://localhost:8081/healthkit and
  similarly I have an externally deployed version at https://my-healthkit-data.fly.dev
  - I will then run queries against this database to build the cache file (or possibly a smaller custom sqlite file)

## TODO
* This is still a work in progress
* Need to write the queries to marshall the data for each of the workouts within each group of walks
* Then cache this data - maybe try another (small) sqlite db for the caching (instead of feather)
* **NOTE: It looks like the queries are being truncated at 1000 values - need to fix**


In [12]:
#export
import os
import pandas as pd
import activityio as aio
from dateutil.parser import parse
import datetime as dt
import sqlite3 as sql
from pathlib import Path
import tomli
import klib


In [3]:
db_file = Path('emmaus_walking.sqlite.db')

In [4]:
LOCAL_DB_URL = 'http://localhost:8081/'
HOSTED_DB_URL = 'https://my-healthkit-data.fly.dev/'

In [5]:
url_CSV = 'http://localhost:8081/healthkit.csv?sql=select%0D%0A++id%2C%0D%0A++workoutActivityType%2C%0D%0A++duration%2C%0D%0A++durationUnit%2C%0D%0A++totalDistance%2C%0D%0A++totalDistanceUnit%2C%0D%0A++totalEnergyBurned%2C%0D%0A++totalEnergyBurnedUnit%2C%0D%0A++sourceName%2C%0D%0A++sourceVersion%2C%0D%0A++creationDate%2C%0D%0A++startDate%2C%0D%0A++endDate%2C%0D%0A++metadata_HKTimeZone%2C%0D%0A++metadata_HKWeatherTemperature%2C%0D%0A++metadata_HKWeatherHumidity%2C%0D%0A++device%2C%0D%0A++metadata_HKElevationAscended%2C%0D%0A++metadata_HKAverageMETs%0D%0Afrom%0D%0A++workouts%0D%0Aorder+by%0D%0A++id%0D%0Alimit%0D%0A++101'

In [6]:
url_CSV2 = 'http://localhost:8081/healthkit.csv?sql=select%0D%0A++id%2C%0D%0A++workoutActivityType%2C%0D%0A++duration%2C%0D%0A++durationUnit%2C%0D%0A++totalDistance%2C%0D%0A++totalDistanceUnit%2C%0D%0A++totalEnergyBurned%2C%0D%0A++totalEnergyBurnedUnit%2C%0D%0A++sourceName%2C%0D%0A++sourceVersion%2C%0D%0A++creationDate%2C%0D%0A++startDate%2C%0D%0A++endDate%2C%0D%0A++metadata_HKTimeZone%2C%0D%0A++workout_events%2C%0D%0A++metadata_HKWeatherTemperature%2C%0D%0A++metadata_HKWeatherHumidity%2C%0D%0A++device%2C%0D%0A++metadata_HKElevationAscended%2C%0D%0A++metadata_HKAverageMETs%2C%0D%0A++metadata_HKMaximumSpeed%2C%0D%0A++metadata_HKAverageSpeed%0D%0Afrom%0D%0A++workouts%0D%0Awhere%0D%0A++workoutActivityType+in+%28%3Ap0%2C+%3Ap1%29%0D%0Aorder+by%0D%0A++creationDate%0D%0Alimit%0D%0A++101&p0=HKWorkoutActivityTypeWalking&p1=HKWorkoutActivityTypeHiking&_size=max'

In [7]:
workouts_df = pd.read_csv(url_CSV2)

#print((LOCAL_DB_URL + 'workout.json'))
#workout_df = pd.read_json(LOCAL_DB_URL + 'workouts.json')

In [8]:
workouts_df.nunique()

id                               101
workoutActivityType                1
duration                         101
durationUnit                       1
totalDistance                    101
totalDistanceUnit                  1
totalEnergyBurned                101
totalEnergyBurnedUnit              1
sourceName                         1
sourceVersion                     10
creationDate                     101
startDate                        101
endDate                          101
metadata_HKTimeZone                3
workout_events                    48
metadata_HKWeatherTemperature     33
metadata_HKWeatherHumidity        51
device                             3
metadata_HKElevationAscended      22
metadata_HKAverageMETs             0
metadata_HKMaximumSpeed            0
metadata_HKAverageSpeed            0
dtype: int64

In [11]:
workouts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             101 non-null    object 
 1   workoutActivityType            101 non-null    object 
 2   duration                       101 non-null    float64
 3   durationUnit                   101 non-null    object 
 4   totalDistance                  101 non-null    float64
 5   totalDistanceUnit              101 non-null    object 
 6   totalEnergyBurned              101 non-null    float64
 7   totalEnergyBurnedUnit          101 non-null    object 
 8   sourceName                     101 non-null    object 
 9   sourceVersion                  101 non-null    object 
 10  creationDate                   101 non-null    object 
 11  startDate                      101 non-null    object 
 12  endDate                        101 non-null    obj

In [13]:
workouts_clean_df = klib.data_cleaning(workouts_df)

Long column names detected (>25 characters). Consider renaming the following columns ['metadata_hkweather_temperature', 'metadata_hkweather_humidity', 'metadata_hkelevation_ascended'].
Shape of cleaned data: (101, 14)Remaining NAs: 182

Changes:
Dropped rows: 0
     of which 0 duplicates. (Rows: [])
Dropped columns: 8
     of which 5 single valued.     Columns: ['workout_activity_type', 'duration_unit', 'total_distance_unit', 'total_energy_burned_unit', 'source_name']
Dropped missing values: 303
Reduced memory by at least: 0.01 MB (-50.0%)

/opt/homebrew/Caskroom/mambaforge/base/envs/emmaus_walking_py38/lib/python3.9/site-packages/klib/clean.py:76: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns.str.replace("\n", "_")


In [14]:
workouts_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 0 to 100
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              101 non-null    string 
 1   duration                        101 non-null    Float32
 2   total_distance                  101 non-null    Float32
 3   total_energy_burned             101 non-null    Float32
 4   source_version                  101 non-null    string 
 5   creation_date                   101 non-null    string 
 6   start_date                      101 non-null    string 
 7   end_date                        101 non-null    string 
 8   metadata_hktime_zone            90 non-null     string 
 9   workout_events                  101 non-null    string 
 10  metadata_hkweather_temperature  88 non-null     string 
 11  metadata_hkweather_humidity     88 non-null     string 
 12  device                          35 n

In [9]:
workouts_df.head()

,id,workoutActivityType,duration,durationUnit,totalDistance,totalDistanceUnit,totalEnergyBurned,totalEnergyBurnedUnit,sourceName,sourceVersion,...,endDate,metadata_HKTimeZone,workout_events,metadata_HKWeatherTemperature,metadata_HKWeatherHumidity,device,metadata_HKElevationAscended,metadata_HKAverageMETs,metadata_HKMaximumSpeed,metadata_HKAverageSpeed
0,a34036ff616122952fa67c9bc11a493f8642dd7c,HKWorkoutActivityTypeWalking,65.506630,min,6.740715,km,2484.388072,kJ,Michael and Ai Leen’s Apple Watch,3.2.3,...,2017-07-31 11:58:05 +1000,Australia/Sydney,"[{""type"": ""HKWorkoutEventTypePause"", ""date"": ""...",61 degF,7300 %,NaN,NaN,NaN,NaN,NaN
1,190a107e28ccac99b6cb2c773577f78db12c8076,HKWorkoutActivityTypeWalking,18.709657,min,1.551418,km,615.905720,kJ,Michael and Ai Leen’s Apple Watch,3.2.3,...,2017-08-01 07:12:09 +1000,Australia/Sydney,[],49 degF,7300 %,NaN,NaN,NaN,NaN,NaN
2,c0e18387229d7b4c2aebdf1131be66e82a35f5aa,HKWorkoutActivityTypeWalking,80.841812,min,7.720215,km,2413.812360,kJ,Michael and Ai Leen’s Apple Watch,3.2.3,...,2017-08-07 10:12:27 +1000,Australia/Sydney,"[{""type"": ""HKWorkoutEventTypePause"", ""date"": ""...",64 degF,3900 %,NaN,NaN,NaN,NaN,NaN
3,b6cc7e549f2dc5c100b1924da88a1a26decb63ad,HKWorkoutActivityTypeWalking,37.775582,min,2.479298,km,783.930976,kJ,Michael and Ai Leen’s Apple Watch,3.2.3,...,2017-08-12 08:52:10 +1000,Australia/Sydney,"[{""type"": ""HKWorkoutEventTypePause"", ""date"": ""...",59 degF,4300 %,NaN,NaN,NaN,NaN,NaN
4,c157a93d6230da905b314ed333c4e3fbe4be1ed0,HKWorkoutActivityTypeWalking,84.599802,min,7.844746,km,2178.930968,kJ,Michael and Ai Leen’s Apple Watch,3.2.3,...,2017-08-26 09:07:08 +1000,Australia/Sydney,"[{""type"": ""HKWorkoutEventTypeMarker"", ""date"": ...",54 degF,6300 %,NaN,NaN,NaN,NaN,NaN


In [9]:
workouts_fly_df = pd.read_csv(url_CSV.replace(LOCAL_DB_URL, HOSTED_DB_URL))

In [10]:
workouts_fly_df.head()

,id,workoutActivityType,duration,durationUnit,totalDistance,totalDistanceUnit,totalEnergyBurned,totalEnergyBurnedUnit,sourceName,sourceVersion,creationDate,startDate,endDate,metadata_HKTimeZone,metadata_HKWeatherTemperature,metadata_HKWeatherHumidity,device,metadata_HKElevationAscended,metadata_HKAverageMETs
0,002a7fbcc124ce02c76554f8d505a2e0555f906e,HKWorkoutActivityTypeWalking,122.173776,min,3.674600,km,1729.920665,kJ,Michael and Ai Leen’s Apple Watch,6.2.6,2020-06-27 08:34:30 +1000,2020-06-27 06:32:18 +1000,2020-06-27 08:34:28 +1000,Australia/Sydney,49 degF,8500 %,"<<HKDevice: 0x282f6c280>, name:Apple Watch, ma...",4630 cm,2.74876 kcal/hr·kg
1,00a2dae192724c8c1940c393a8bfef79d37cbdbe,HKWorkoutActivityTypeWalking,117.242568,min,1.803008,km,1289.166333,kJ,Michael and Ai Leen’s Apple Watch,6.2.1,2020-06-13 15:13:35 +1000,2020-06-13 13:16:17 +1000,2020-06-13 15:13:32 +1000,Australia/Sydney,67 degF,7000 %,"<<HKDevice: 0x282f600a0>, name:Apple Watch, ma...",2803 cm,2.25918 kcal/hr·kg
2,0121372783b05b67ef82219fde21347b87636b5b,HKWorkoutActivityTypeCrossTraining,7.883268,min,0.000000,km,513.001475,kJ,Seven,420,2016-07-20 08:42:54 +1000,2016-07-20 08:35:00 +1000,2016-07-20 08:42:54 +1000,Australia/Sydney,NaN,NaN,NaN,NaN,NaN
3,0123895b3103842c96484bd5baeeca3e57481ce5,HKWorkoutActivityTypeWalking,103.204779,min,3.650011,km,1544.586192,kJ,Michael and Ai Leen’s Apple Watch,7.1,2020-12-14 09:33:44 +1000,2020-12-14 07:50:29 +1000,2020-12-14 09:33:41 +1000,Australia/Sydney,69.8 degF,8200 %,"<<HKDevice: 0x282f745a0>, name:Apple Watch, ma...",4271 cm,2.88149 kcal/hr·kg
4,013404c2aaf1f1e14314e756a0edb5ae54d57397,HKWorkoutActivityTypeHiking,57.604634,min,1.218353,km,469.549308,kJ,Michael and Ai Leen’s Apple Watch,5.1.3,2019-04-12 10:18:27 +1000,2019-04-12 09:20:30 +1000,2019-04-12 10:18:06 +1000,Australia/Sydney,65 degF,7500 %,"<<HKDevice: 0x282fdf110>, name:Apple Watch, ma...",1607 cm,NaN


In [11]:
workout_points_SQL = 'http://localhost:8081/healthkit.csv?sql=select%0D%0A++rowid%2C%0D%0A++date%2C%0D%0A++latitude%2C%0D%0A++longitude%2C%0D%0A++altitude%2C%0D%0A++speed%0D%0Afrom%0D%0A++workout_points%0D%0Awhere%0D%0A++workout_id+%3D+%22'
workout_id = 'a34036ff616122952fa67c9bc11a493f8642dd7c' + '%22'

workout_points_df = pd.read_csv(workout_points_SQL + workout_id, parse_dates=True)

In [12]:
workout_points_df.head()

,rowid,date,latitude,longitude,altitude,speed
0,1,2017-07-31T00:51:19Z,-33.857455,151.160325,-4.943426,2.993684
1,2,2017-07-31T00:52:08Z,-33.857726,151.160033,20.688181,1.673957
2,3,2017-07-31T00:52:09Z,-33.857726,151.160052,20.902155,1.677370
3,4,2017-07-31T00:52:10Z,-33.857727,151.160071,21.125973,1.682297
4,5,2017-07-31T00:52:11Z,-33.857727,151.160087,21.315756,1.687898


In [10]:
workout_points_df.info()

NameError: name 'workout_points_df' is not defined

In [14]:
WALK_DETAILS_FILE = 'walk_details.toml'
walk_details = Path('../' + WALK_DETAILS_FILE)


In [15]:
walk_details

Path('../walk_details.toml')

In [16]:
with open(walk_details, encoding="utf-8") as f:
    walk_details_dict = tomli.load(f)

In [17]:
walk_details_dict

{'walks': [{'short_name': 'B2M', 'name': 'Bondi to Manly'},
  {'short_name': 'B2W', 'name': 'Bondi to Wollongong'},
  {'short_name': 'D2C', 'name': 'Drummoyne to Cockatoo'},
  {'short_name': 'GNW', 'name': 'Great North Walk'},
  {'short_name': 'GTL', 'name': 'Gladesville Loop'},
  {'short_name': 'GNW', 'name': 'Great North Walk'},
  {'short_name': 'GWW', 'name': 'Great West Walk', 'status': 'incomplete'},
  {'short_name': 'OLD', 'name': 'Old Bar'},
  {'short_name': 'STM', 'name': "St Michael's Golf Course"},
  {'short_name': 'SNM', 'name': 'Snowy Mountains (Thredo)'},
  {'short_name': 'WNG',
   'name': 'Newcastle to Sydney',
   'status': 'incomplete'}]}

In [18]:
pd.DataFrame(walk_details_dict, )

,walks
0,"{'short_name': 'B2M', 'name': 'Bondi to Manly'}"
1,"{'short_name': 'B2W', 'name': 'Bondi to Wollon..."
2,"{'short_name': 'D2C', 'name': 'Drummoyne to Co..."
3,"{'short_name': 'GNW', 'name': 'Great North Walk'}"
4,"{'short_name': 'GTL', 'name': 'Gladesville Loop'}"
5,"{'short_name': 'GNW', 'name': 'Great North Walk'}"
6,"{'short_name': 'GWW', 'name': 'Great West Walk..."
7,"{'short_name': 'OLD', 'name': 'Old Bar'}"
8,"{'short_name': 'STM', 'name': 'St Michael's Go..."
9,"{'short_name': 'SNM', 'name': 'Snowy Mountains..."


In [19]:
#export
def calc_walk_stats(walk_data):
    total_time = dt.timedelta(0)
    total_distance = 0

    for iHike, hike in enumerate(walk_data):
        total_time += hike.index.max()
        # print(iHike+1, walk_date[iHike], hike.index.max(), hike['dist'].max() / 1e3)
        total_distance += hike['dist'].max()
    total_distance /= 1e3

    start_coord = walk_data[0][['lat', 'lon']].iloc[0].tolist()
    end_coord = walk_data[-1][['lat', 'lon']].iloc[-1].tolist()
    return total_time, total_distance, start_coord, end_coord


# TODO: use st.cache() and also look to pre-load and cache/feather data (or similar) - NB: use of @st.cache() below didn't work
def load_and_cache_raw_walk_data(walk_name, sample_freq, conn):
    RAW_FIT_FILE_PATH = 'icloud/Data/HealthFit/FIT' 
    fit_dir = Path.home()/RAW_FIT_FILE_PATH
    data_dir = fit_dir/walk_name[0:3]
    print(data_dir.ls())
    data_files = [file for file in os.listdir(data_dir) if file.endswith('.fit')]
    walk_files = sorted(data_files)
    print(walk_files)

    walk_data = []
    walk_date = []

    for iFile, file in enumerate(walk_files):
        walk_df = pd.DataFrame(aio.read(data_dir + file))
        walk_data.append(walk_df)
        walk_date.append(parse(file[0:17]))
        walk_df['WalkName'] = walk_name
        walk_df['WalkNumber'] = iFile
        walk_df[['alt', 'dist', 'lat', 'lon', 'speed', 'WalkName', 'WalkNumber']].to_sql('walks', conn, if_exists='append', index=False)
               
    total_time, total_distance, start_coord, end_coord = calc_walk_stats(walk_data)
    walk_stats = [total_time, total_distance, start_coord, end_coord]
    #print(start_coord)
    walk_merged = pd.concat(walk_data)
    points = walk_merged[['lat', 'lon']].values.tolist()
    points = [tuple(point) for ipoint, point in enumerate(points) if ipoint % sample_freq == 0]
    return walk_data, walk_date, walk_files, points, walk_stats

In [20]:
def create_walk_cached_data_for_app(db_file, n_rows_used=5):
    # read in all of the walks data and sample at an appropriate frequency and cache for faster use in the app
    db_conn = sql.connect(db_file)
    walk_df = pd.read_sql_query('SELECT * FROM walks', db_conn)

    UNUSED_COLUMNS = ['dist', 'speed']

    walk_df.drop(UNUSED_COLUMNS, axis=1, inplace=True)
    walk_df.dropna(inplace=True)      # TODO: Check why there are a few NaNs
    walk_df = walk_df.iloc[::n_rows_used].reset_index()    # downsample

    walk_df.to_feather(Path(db_file.as_posix().replace('.db', '.cache.feather')))
    
    return walk_df

In [21]:
# Not working yet -- this is the alternate approach to using the individual .FIT files
# walk_df = create_walk_cached_data_for_app(db_file, 10)

In [22]:
# walk_df[walk_df['lat'].isna()]

In [23]:
Path(db_file.as_posix().replace('.db', '.cache.feather'))

Path('emmaus_walking.sqlite.cache.feather')

In [24]:
# walk_df = pd.read_feather(Path(db_file.as_posix().replace('.db', '.cache.feather')))

In [25]:
# walk_df.info()

In [26]:
# walk_df['WalkName'].unique()